#Initialization:

In [59]:
import pandas as pd

import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path1 = '/content/drive/MyDrive/VU/project_flatlands/flatland_train.npz'

Mounted at /content/drive


In [51]:
data = np.load(path1)
X = data['X']
y = data['y']

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]
X = X.reshape(-1, 50, 50, 1)  # Reshape X to have a channel dimension

In [52]:
print(X.shape)
print(X[1].shape)
print(y.shape)

(10000, 50, 50, 1)
(50, 50, 1)
(10000,)


In [53]:
min_value = y.min()
max_value = y.max()
print(min_value)
print(max_value)

0.0
4.0


GPU set up properly?:

In [24]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


#Data Preparation:

In [54]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
y_train.shape

(8000,)

# Model:

In [62]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=[50, 50, 1]))
model.add(BatchNormalization(axis=-1))   # Axis -1 is always the features axis
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=-1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_6 (Bat  (None, 48, 48, 32)        128       
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 48, 48, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 46, 46, 32)        9248      
                                                                 
 batch_normalization_7 (Bat  (None, 46, 46, 32)        128       
 chNormalization)                                                
                                                                 
 activation_29 (Activation)  (None, 46, 46, 32)      

#Train loop:

In [75]:
# Train the model (Adjust batch_size and epochs as per your requirement)
model.fit(X_train, y_train, batch_size=32, epochs=12)

Epoch 1/12
250/250 [==============================] - 3s 7ms/step - loss: 0.0048 - accuracy: 0.9985
Epoch 2/12
250/250 [==============================] - 2s 6ms/step - loss: 0.0035 - accuracy: 0.9987
Epoch 3/12
250/250 [==============================] - 2s 6ms/step - loss: 0.0148 - accuracy: 0.9965
Epoch 4/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0106 - accuracy: 0.9964
Epoch 5/12
250/250 [==============================] - 2s 6ms/step - loss: 0.0055 - accuracy: 0.9980
Epoch 6/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0074 - accuracy: 0.9983
Epoch 7/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0024 - accuracy: 0.9995
Epoch 8/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0015 - accuracy: 0.9994
Epoch 9/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0071 - accuracy: 0.9977
Epoch 10/12
250/250 [==============================] - 2s 7ms/step - loss: 0.0037 - accuracy: 0.9992

In [76]:
# Save the model
model.save('model_keras.h5')

#Test model:

In [77]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model('model_keras.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

63/63 [==============================] - 0s 4ms/step - loss: 0.5346 - accuracy: 0.9595
Test accuracy: 0.9595000147819519
